# Save Data

In [1]:
import pandas as pd
import numpy as np
import torch
import torchvision.transforms as tt
from PIL import Image
import glob
from tqdm import tqdm
# dd

## label (Y)
- Y_train (16008 rows × 63 columns)
- Y_test (1596 rows × 63 columns)

In [2]:
annotation_train = pd.read_csv('data/Training/Annotation_Training.csv')
Y_train = annotation_train.iloc[3:, :]
Y_train.columns = range(Y_train.shape[1])
Y_train = Y_train.set_index(0)

annotation_test = pd.read_csv('data/Testing/Annotation_Testing.csv')
Y_test = annotation_test.iloc[3:, :]
Y_test.columns = range(Y_test.shape[1])
Y_test = Y_test.set_index(0)

info = annotation_train.iloc[:1, :6]  # train, test 같음
info


,focal_length_x,focal_length_y,res_x,res_y,tile_x,tile_y
0,0.7531,1.004,320.0,240.0,0.0,0.0


## image (X)
- X_train
    - depth_1_0000001.png ~ depth_1_0016008.png
- X_test
    - depth_1_0000001.png ~ depth_1_0001596.png

In [3]:
path_train = 'data/Training/depth/*.png'
path_test = 'data/Testing/depth/*.png'

transform = tt.Compose([
    tt.PILToTensor()
])


In [4]:
img_list = []
for filename in tqdm(glob.glob(path_train)):
    img = Image.open(filename)
    img_tensor = transform(img)  # torch.Size([1, 240, 320])
    img_list.append(img_tensor)
train_data = torch.stack(img_list)  # torch.Size([data_len, 1, 240, 320])
torch.save(train_data, 'data/Training/train_data.pt')

img_list = []
for filename in tqdm(glob.glob(path_test)):
    img = Image.open(filename)
    img_tensor = transform(img)  # torch.Size([1, 240, 320])
    img_list.append(img_tensor)
test_data = torch.stack(img_list)  # torch.Size([data_len, 1, 240, 320])
torch.save(test_data, 'data/Testing/test_data.pt')


100%|██████████| 1596/1596 [00:02<00:00, 663.29it/s]


In [5]:
img_list = []
for filename in tqdm(glob.glob(path_train)):
    img = Image.open(filename)
    img_tensor = transform(img)  # torch.Size([1, 240, 320])

    ###
    c_min = img_tensor.min()
    c_max = img_tensor.max()
    if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
        img_tensor = img_tensor.to(torch.int8)
    elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
        img_tensor = img_tensor.to(torch.int16)
    elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
        img_tensor = img_tensor.to(torch.int32)
    elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
        img_tensor = img_tensor.to(torch.int64)
    ###

    img_list.append(img_tensor)
train_data = torch.stack(img_list)  # torch.Size([data_len, 1, 240, 320])
torch.save(train_data, 'data/Training/train_data_diet.pt')


100%|██████████| 16008/16008 [00:26<00:00, 596.53it/s]


In [6]:
img_list = []
for filename in tqdm(glob.glob(path_test)):
    img = Image.open(filename)
    img_tensor = transform(img)  # torch.Size([1, 240, 320])

    ###
    c_min = img_tensor.min()
    c_max = img_tensor.max()
    if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
        img_tensor = img_tensor.to(torch.int8)
    elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
        img_tensor = img_tensor.to(torch.int16)
    elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
        img_tensor = img_tensor.to(torch.int32)
    elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
        img_tensor = img_tensor.to(torch.int64)
    ###

    img_list.append(img_tensor)
test_data = torch.stack(img_list)  # torch.Size([data_len, 1, 240, 320])
torch.save(test_data, 'data/Testing/test_data_diet.pt')


100%|██████████| 1596/1596 [00:02<00:00, 586.02it/s]


# Load Data

In [8]:
train_data = torch.load('data/Training/train_data_diet.pt')
test_data = torch.load('data/Testing/test_data_diet.pt') 